In [58]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-m02j7ga3
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-m02j7ga3
  Resolved https://github.com/ibm-granite-community/utils to commit 1514191fbbc4605ed4fdfdcb448f2ee41477058f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [59]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":2048, "temperature":0.2},
)

In [60]:
def fewshot_prompt(context, question, ui_components, examples):
    titles = ", ".join(ui_components)

    formatted_examples = "\n\n".join(
        f"""Example {i+1}:
User Question: {example['question']}
Context: {example['context']}
Model Output: {example['output']}""" for i, example in enumerate(examples)
    )

    prompt = f"""
You are an expert in creating clean and interactive UIs for mental health tools using Python ipywidgets.
Design a Jupyter Notebook UI component with the following goal:

{formatted_examples}

Now complete this new request:

Context: {context}
User Question: {question}
Include elements such as: {titles}
Make sure to:
- Use ipywidgets
- Keep layout clean and centered
- Combine all sections into a VBox
- Output only the Python code
"""
    return prompt

In [61]:
# Function to get answer using few-shot prompting
def get_answer_using_fewshot(context, question, ui_components, examples):
    """
    Generates the response from the model based on a few-shot prompt.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question for the model to answer
    - book_titles: list, list of book titles to include in the prompt

    Returns:
    - str, the generated result from the model
    """
    prompt = fewshot_prompt(context, question, ui_components, examples)
    result = model.invoke(prompt)

    return result

In [62]:
examples = [
    {
        "question": "Add a styled header and welcome message for my mood tracker app",
        "context": "Minimalistic mood tracker header and subtext using gradient background",
        "output": """
import ipywidgets as widgets
from IPython.display import display

header = widgets.HTML(value=\"\"\"
<div style='background: linear-gradient(to right, #2193b0, #6dd5ed); padding: 20px; border-radius: 8px;'>
    <h1 style='color: white; text-align: center; font-family: Arial;'>Daily Mood Tracker</h1>
</div>
\"\"\")
welcome = widgets.HTML(value=\"\"\"
<div style='padding: 15px;'>
    <h3 style='text-align: center;'>Welcome! How are you feeling today?</h3>
    <p style='text-align: center; color: #555;'>Log your daily mood and reflect on your emotional wellbeing 🌈</p>
</div>
\"\"\")
display(widgets.VBox([header, welcome]))
"""
    },
    {
        "question": "Add mood dropdown and note input section",
        "context": "Interactive form using ipywidgets with dropdown and textarea",
        "output": """
import ipywidgets as widgets
from IPython.display import display

mood_dropdown = widgets.Dropdown(
    options=['😊 Happy', '😐 Neutral', '😢 Sad', '😠 Angry', '😰 Anxious', '😴 Tired'],
    description='Mood:',
    layout=widgets.Layout(width='50%')
)

note_input = widgets.Textarea(
    placeholder='Add a short note (optional)...',
    description='Note:',
    layout=widgets.Layout(width='90%', height='80px')
)

submit_button = widgets.Button(
    description='Submit Mood',
    button_style='info',
    layout=widgets.Layout(width='30%')
)

form = widgets.VBox([mood_dropdown, note_input, submit_button], layout=widgets.Layout(align_items='center'))
display(form)
"""
    },
    {
        "question": "Create full daily mood tracker UI with input and history",
        "context": "Complete mood tracker page layout with submission handler and mood history output",
        "output": """
import ipywidgets as widgets
from IPython.display import display

# Header
header = widgets.HTML(value=\"\"\"
<div style='background: linear-gradient(to right, #2193b0, #6dd5ed); padding: 20px; border-radius: 8px;'>
    <h1 style='color: white; text-align: center; font-family: Arial;'>Daily Mood Tracker</h1>
</div>
\"\"\")
# Welcome
welcome = widgets.HTML(value=\"\"\"
<div style='padding: 15px;'>
    <h3 style='text-align: center;'>Welcome! How are you feeling today?</h3>
    <p style='text-align: center; color: #555;'>Log your daily mood and reflect on your emotional wellbeing 🌈</p>
</div>
\"\"\")
# Inputs
mood_dropdown = widgets.Dropdown(
    options=['😊 Happy', '😐 Neutral', '😢 Sad', '😠 Angry', '😰 Anxious', '😴 Tired'],
    description='Mood:',
    layout=widgets.Layout(width='50%')
)

note_input = widgets.Textarea(
    placeholder='Add a short note (optional)...',
    description='Note:',
    layout=widgets.Layout(width='90%', height='80px')
)

submit_button = widgets.Button(
    description='Submit Mood',
    button_style='info',
    layout=widgets.Layout(width='30%', align_self='center')
)

output = widgets.Output()
mood_entries = []

def handle_submit(_):
    mood = mood_dropdown.value
    note = note_input.value
    mood_entries.append((mood, note))
    with output:
        output.clear_output()
        display(widgets.HTML("<h4 style='color: #333;'>Mood History</h4>"))
        for i, (m, n) in enumerate(reversed(mood_entries)):
            display(widgets.HTML(f\"\"\"
                <div style='background-color: #f9f9f9; padding: 10px; margin: 5px; border-radius: 5px;'>
                    <b>Entry {len(mood_entries) - i}:</b> {m}<br>
                    <i>Note:</i> {n if n.strip() else "-"}
                </div>
            \"\"\"))

    note_input.value = ''

submit_button.on_click(handle_submit)

form = widgets.VBox([mood_dropdown, note_input, submit_button], layout=widgets.Layout(align_items='center'))
page = widgets.VBox([header, welcome, form, output])
display(page)
"""
    }
]

In [63]:
# Example usage
context = "Design and develop a daily mood tracker landing page using ipywidgets with interactive features"
question = "Create the landing page with header, welcome message, mood dropdown, note input, submit button, and history output section in VBox"
ui_components = ["Header", "Welcome message", "Mood dropdown", "Note input", "Submit button", "History output"]
# Generate and display the UI code for the landing page
result = get_answer_using_fewshot(context, question, ui_components, examples)
print(f"Generated Code:\n{result}")

Generated Code:
```python
import ipywidgets as widgets
from IPython.display import display

# Header
header = widgets.HTML(value="""
<div style='background: linear-gradient(to right, #2193b0, #6dd5ed); padding: 20px; border-radius: 8px;'>
    <h1 style='color: white; text-align: center; font-family: Arial;'>Daily Mood Tracker</h1>
</div>
""")

# Welcome
welcome = widgets.HTML(value="""
<div style='padding: 15px;'>
    <h3 style='text-align: center;'>Welcome! How are you feeling today?</h3>
    <p style='text-align: center; color: #555;'>Log your daily mood and reflect on your emotional wellbeing 🌈</p>
</div>
""")

# Mood Dropdown
mood_dropdown = widgets.Dropdown(
    options=['😊 Happy', '😐 Neutral', '😢 Sad', '😠 Angry', '😰 Anxious', '😴 Tired'],
    description='Mood:',
    layout=widgets.Layout(width='50%')
)

# Note Input
note_input = widgets.Textarea(
    placeholder='Add a short note (optional)...',
    description='Note:',
    layout=widgets.Layout(width='90%', height='80px')
)

# S

In [79]:
import ipywidgets as widgets
from IPython.display import display

# Header
header = widgets.HTML(value="""
<div style='background: linear-gradient(to right, #2193b0, #6dd5ed); padding: 20px; border-radius: 8px;'>
    <h1 style='color: white; text-align: center; font-family: Arial;'>Daily Mood Tracker</h1>
</div>
""")

# Welcome
welcome = widgets.HTML(value="""
<div style='padding: 15px;'>
    <h3 style='text-align: center;'>Welcome! How are you feeling today?</h3>
    <p style='text-align: center; color: #555;'>Log your daily mood and reflect on your emotional wellbeing 🌈</p>
</div>
""")

# Mood Dropdown
mood_dropdown = widgets.Dropdown(
    options=['😊 Happy', '😐 Neutral', '😢 Sad', '😠 Angry', '😰 Anxious', '😴 Tired'],
    description='Mood:',
    layout=widgets.Layout(width='50%')
)

# Note Input
note_input = widgets.Textarea(
    placeholder='Add a short note (optional)...',
    description='Note:',
    layout=widgets.Layout(width='90%', height='80px')
)

# Submit Button
submit_button = widgets.Button(
    description='Submit Mood',
    button_style='info',
    layout=widgets.Layout(width='30%', align_self='center')
)

# Output for Mood History
output = widgets.Output()
mood_entries = []

def refresh_output():
    output.clear_output()
    with output:
        display(widgets.HTML("<h4 style='color: #333; text-align: center;'>Mood History</h4>"))
        for i, (m, n) in enumerate(reversed(mood_entries)):
            idx = len(mood_entries) - 1 - i  # index asli di list

            delete_button = widgets.Button(description="❌", layout=widgets.Layout(width='40px'))

            def make_delete_handler(index):
                def delete(_):
                    mood_entries.pop(index)
                    refresh_output()
                return delete

            delete_button.on_click(make_delete_handler(idx))

            entry_box = widgets.HBox([
                widgets.HTML(value=f"""
                    <div style='background-color: #e3f2fd; padding: 12px; border-radius: 8px; border: 1px solid #90caf9;
                                 box-shadow: 0 2px 4px rgba(0,0,0,0.05); width: 100%;'>
                        <b style='color: #0d47a1;'>Entry {len(mood_entries) - i}:</b> {m}<br>
                        <span style='color: #333;'><i>Note:</i> {n if n.strip() else "-"}</span>
                    </div>
                """),
                delete_button
            ])

            display(entry_box)

def handle_submit(_):
    mood = mood_dropdown.value
    note = note_input.value
    mood_entries.append((mood, note))
    note_input.value = ''
    refresh_output()

submit_button.on_click(handle_submit)

# Combine all sections into a VBox
form = widgets.VBox([mood_dropdown, note_input, submit_button], layout=widgets.Layout(align_items='center'))
page = widgets.VBox([header, welcome, form, output], layout=widgets.Layout(align_items='center'))

display(page)

In [80]:
def refine_output_with_fewshot(previous_output, context, question, ui_components, examples):
    """
    Refines a previously generated output by applying additional user-requested changes
    using a few-shot prompt technique.

    Parameters:
    - previous_output: str, the code generated from the initial prompt.
    - context: str, the context for the refinement task.
    - question: str, the user-requested changes to be applied.
    - book_titles: list, list of UI component names for better grounding.
    - examples: list, formatted examples of similar tasks for few-shot learning.

    Returns:
    - str, the refined Python code generated by the model.
    """

    formatted_examples = "\n\n".join(
        f"""
        Example {i+1}:
        User Question: {example['question']}
        Context: {example['context']}
        Model Output: {example['output']}
        """
        for i, example in enumerate(examples)
    )

    prompt = f"""
You are an experienced Python developer with expertise in building interactive UIs in Jupyter Notebooks using ipywidgets.

Here are some tasks you have done before:
{formatted_examples}

Now, refine the following Python UI code to match the latest user request.

Current Output:
{previous_output}

Context:
{context}

User Request:
{question}

Only return the complete Python code.
"""

    return model.invoke(prompt)


In [81]:
# Example usage
refine_context = "Add hover effects and rounded borders to mood history cards, and center align the header and welcome message."
refine_question = "Improve UI aesthetics with hover animations and soft box shadows for each mood entry."
refined_result = refine_output_with_fewshot(result, refine_context, refine_question, ui_components, examples)

print(f"Refined Output:\n{refined_result}")

Refined Output:

```python
import ipywidgets as widgets
from IPython.display import display

# Header
header = widgets.HTML(value="""
<div style='background: linear-gradient(to right, #2193b0, #6dd5ed); padding: 20px; border-radius: 8px; text-align: center;'>
    <h1 style='color: white; font-family: Arial;'>Daily Mood Tracker</h1>
</div>
""")

# Welcome
welcome = widgets.HTML(value="""
<div style='padding: 15px; text-align: center;'>
    <h3>Welcome! How are you feeling today?</h3>
    <p style='color: #555;'>Log your daily mood and reflect on your emotional wellbeing 🌈</p>
</div>
""")

# Mood Dropdown
mood_dropdown = widgets.Dropdown(
    options=['😊 Happy', '😐 Neutral', '😢 Sad', '😠 Angry', '😰 Anxious', '😴 Tired'],
    description='Mood:',
    layout=widgets.Layout(width='50%')
)

# Note Input
note_input = widgets.Textarea(
    placeholder='Add a short note (optional)...',
    description='Note:',
    layout=widgets.Layout(width='90%', height='80px')
)

# Submit Button
submit_button 

In [82]:
import ipywidgets as widgets
from IPython.display import display

# Header
header = widgets.HTML(value="""
<div style='background: linear-gradient(to right, #2193b0, #6dd5ed); padding: 20px; border-radius: 8px; text-align: center;'>
    <h1 style='color: white; font-family: Arial;'>Daily Mood Tracker</h1>
</div>
""")

# Welcome
welcome = widgets.HTML(value="""
<div style='padding: 15px; text-align: center;'>
    <h3>Welcome! How are you feeling today?</h3>
    <p style='color: #555;'>Log your daily mood and reflect on your emotional wellbeing 🌈</p>
</div>
""")

# Mood Dropdown
mood_dropdown = widgets.Dropdown(
    options=['😊 Happy', '😐 Neutral', '😢 Sad', '😠 Angry', '😰 Anxious', '😴 Tired'],
    description='Mood:',
    layout=widgets.Layout(width='50%')
)

# Note Input
note_input = widgets.Textarea(
    placeholder='Add a short note (optional)...',
    description='Note:',
    layout=widgets.Layout(width='90%', height='80px')
)

# Submit Button
submit_button = widgets.Button(
    description='Submit Mood',
    button_style='info',
    layout=widgets.Layout(width='30%', align_self='center')
)

# Output for Mood History
output = widgets.Output()
mood_entries = []

def handle_submit(_):
    mood = mood_dropdown.value
    note = note_input.value
    mood_entries.append((mood, note))
    with output:
        output.clear_output()
        display(widgets.HTML("<h4 style='color: #333;'>Mood History</h4>"))
        for i, (m, n) in enumerate(reversed(mood_entries)):
            display(widgets.HTML(f"""
                <div style='background-color: #f9f9f9; padding: 10px; margin: 5px; border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); transition: all 0.3s ease; cursor: pointer;'>
                    <b>Entry {len(mood_entries) - i}:</b> {m}<br>
                    <i>Note:</i> {n if n.strip() else "-"}
                </div>
            """))
    note_input.value = ''

submit_button.on_click(handle_submit)

# Combine all sections into a VBox
form = widgets.VBox([mood_dropdown, note_input, submit_button], layout=widgets.Layout(align_items='center'))
page = widgets.VBox([header, welcome, form, output], layout=widgets.Layout(align_items='center'))

display(page)